In [ ]:
import pandas as pd
from datetime import datetime

In [133]:
zeca_carteira = pd.read_excel("./Portifólio_Consolidado.xlsx")

## MONTANDO FUNÇÔES DE POSIÇÃO

In [134]:
def generate_trade(stock_quant, stock_price, action):

    return {
        'shares': stock_quant,
        'price': stock_price,
        'action': action
    }

In [135]:
def generate_fifo(security, actions, quantity, price, data = 0):

    # Portifolio é um dict. As chaves são os tickers dos ativos.
    # Os valores são listas que contém dicts, sendo cada dict um trade do referente ativo
    portifolio = {}

    for i in range(0, len(security)):

        # Pegando cada valor da lista "x" na posição "i"
        ticker = str(security[i])
        action = str(actions[i])
        stock_quant = float(quantity[i])
        stock_price = float(price[i])

        # Gerando um trade com os valores atribuídos
        trade = generate_trade(stock_quant, stock_price, action)
        

        if action == "Compra":
            # Pega a chave igual ticker do portifólio e atribuí ao active_trades
            active_trades = portifolio.get(ticker)
            # Se não houver nenhuma chave com esse ticker no active_trades, acrescente o ticker e trade ao portifolio
            # Se já houver uma chave com esse ticker no active_trades, acrescenta o trade só no active_trades
            if active_trades is None:
                portifolio['{}'.format(ticker)] = [trade]
            else:
                active_trades.append(trade)
        
        if action == "Venda":
            # Pega a chave igual ticker do portifolio e atribuí ao active_trades
            active_trades = portifolio.get(ticker)
            # Define a qtde de casas decimais
            precision = 2
            # Se já existir algum trade para o ativo, a qtde de venda é igual ao valor da chave shares no trade
            if active_trades != None:
                shares_to_sell = float("{:.2f}".format(float(float(trade['shares']))))

                # display(active_trades, security[i] + ' {}'.format(i), shares_to_sell)

                # Garantir que a qtde de venda seja um número
                while shares_to_sell > 0:
                    shares_to_sell = float(shares_to_sell)
                    # Se a qtde de trades ativos > 0, o item de venda será o primeiro trade
                    if len(active_trades) > 0:
                        item_to_sell = active_trades[0]
                        item_to_sell['shares'] = float("{:.2f}".format(float(item_to_sell['shares'])))

                        # Se a qtde de venda = que a qtde do primeiro trade, zero qtde de venda e deleto primeiro trade
                        if item_to_sell['shares'] == shares_to_sell:
                            del active_trades[0]
                            shares_to_sell = 0
                        
                        # Se a qtde de venda > que a qtde do primeiro trade, subtraio a qtde do primeiro trade da qtde de venda e deleto o trade
                        elif item_to_sell['shares'] < shares_to_sell:
                            shares_to_sell -= item_to_sell['shares']
                            del active_trades[0]

                        # se a tde de venda < que a qtde do primeiro trade, subtraio a qtde de venda da qtde do primeiro lote e deleto zero veda
                        else:
                            item_to_sell['shares'] -= shares_to_sell
                            shares_to_sell = 0

                # Se não houver mais trades ativos para o ativo, remove-lo do portifolio
                if len(active_trades) == 0:
                    portifolio.pop(ticker)                    


    return portifolio

In [136]:
def mypositions(security, actions, quantity, price):

    # Gerando portifolio com função fifo
    portifolio = generate_fifo(security, actions, quantity, price)
    resultado = []


    #Percorrer trades de cada ativo
    for key, value in portifolio.items():
        shares = 0
        total_cost = 0
        average_price = 0

        # Percorrendo cada trade, calculando quantidade e preço total
        for i in range(0, len(value)):

            shares += value[i]['shares']
            total_cost += value[i]['shares'] * value[i]['price']

        # Calculando preço médio e atribuindo resultados como lista na lista resultado
        average_price = round((total_cost / shares),2)
        resultado.append([key, shares, average_price])

    # Transformando resutlado em um Data Frame
    carteira = pd.DataFrame(data = resultado, columns=['Ticker', 'Quantidade', 'Preço'])

    return carteira

In [137]:
posições = mypositions(zeca_carteira["Ticker"], zeca_carteira["Operação"], zeca_carteira["Quantidade"], zeca_carteira["Preço"]).sort_values(by='Ticker')

posições = posições.reset_index(drop=True)

posições

,Ticker,Quantidade,Preço
0,B3SA3,350.0,13.86
1,BBDC4,235.0,17.34
2,EGIE3,113.0,42.20
3,ENBR3,198.0,21.99
4,FLRY3,274.0,16.16
5,GGRC11,30.0,112.59
6,HAPV3,332.0,7.24
7,HGLG11,20.0,164.08
8,ITSA4,830.0,9.80
9,JBSS3,90.0,37.03


## Versão 2.0

In [138]:
def generate_trade(stock_quant, stock_price, action):

    return {
        'shares': stock_quant,
        'price': stock_price,
        'action': action
    }

In [139]:
def generate_fifo(security, actions, quantity, price, date = 0):

    # Portifolio é um dict. As chaves são os tickers dos ativos.
    # Os valores são listas que contém dicts, sendo cada dict um trade do referente ativo
    portifolio = {}

    for i in range(0, date if date != 0 else len(security)):

        # Pegando cada valor da lista "x" na posição "i"
        ticker = str(security[i])
        action = str(actions[i])
        stock_quant = float(quantity[i])
        stock_price = float(price[i])

        # Gerando um trade com os valores atribuídos
        trade = generate_trade(stock_quant, stock_price, action)
        

        if action == "Compra":
            # Pega a chave igual ticker do portifólio e atribuí ao active_trades
            active_trades = portifolio.get(ticker)
            # Se não houver nenhuma chave com esse ticker no active_trades, acrescente o ticker e trade ao portifolio
            # Se já houver uma chave com esse ticker no active_trades, acrescenta o trade só no active_trades
            if active_trades is None:
                portifolio['{}'.format(ticker)] = [trade]
            else:
                active_trades.append(trade)
        
        if action == "Venda":
            # Pega a chave igual ticker do portifolio e atribuí ao active_trades
            active_trades = portifolio.get(ticker)
            # Define a qtde de casas decimais
            precision = 2
            # Se já existir algum trade para o ativo, a qtde de venda é igual ao valor da chave shares no trade
            if active_trades != None:
                shares_to_sell = float("{:.2f}".format(float(float(trade['shares']))))

                # display(active_trades, security[i] + ' {}'.format(i), shares_to_sell)

                # Garantir que a qtde de venda seja um número
                while shares_to_sell > 0:
                    shares_to_sell = float(shares_to_sell)
                    # Se a qtde de trades ativos > 0, o item de venda será o primeiro trade
                    if len(active_trades) > 0:
                        item_to_sell = active_trades[0]
                        item_to_sell['shares'] = float("{:.2f}".format(float(item_to_sell['shares'])))

                        # Se a qtde de venda = que a qtde do primeiro trade, zero qtde de venda e deleto primeiro trade
                        if item_to_sell['shares'] == shares_to_sell:
                            del active_trades[0]
                            shares_to_sell = 0
                        
                        # Se a qtde de venda > que a qtde do primeiro trade, subtraio a qtde do primeiro trade da qtde de venda e deleto o trade
                        elif item_to_sell['shares'] < shares_to_sell:
                            shares_to_sell -= item_to_sell['shares']
                            del active_trades[0]

                        # se a tde de venda < que a qtde do primeiro trade, subtraio a qtde de venda da qtde do primeiro lote e deleto zero veda
                        else:
                            item_to_sell['shares'] -= shares_to_sell
                            shares_to_sell = 0

                # Se não houver mais trades ativos para o ativo, remove-lo do portifolio
                if len(active_trades) == 0:
                    portifolio.pop(ticker)                    


    return portifolio

In [140]:
def mypositions(carteira, date = 0):

    # Gerando portifolio com função fifo
    if date != 0:
        if date < carteira.iloc[0]['Data']:
            AttributeError
            exit
        else:
            lenght = len(carteira.loc[carteira['Data'] < date])
    else:
        lenght = len(carteira)    
    
    
    portifolio = generate_fifo(carteira["Ticker"], carteira["Operação"], carteira["Quantidade"], carteira["Preço"], date = lenght)
    resultado = []

    #Percorrer trades de cada ativo
    for key, value in portifolio.items():
        shares = 0
        total_cost = 0
        average_price = 0

        # Percorrendo cada trade, calculando quantidade e preço total
        for i in range(0, len(value)):

            shares += value[i]['shares']
            total_cost += value[i]['shares'] * value[i]['price']

        # Calculando preço médio e atribuindo resultados como lista na lista resultado
        average_price = round((total_cost / shares),2)
        resultado.append([key, shares, average_price])

    # Transformando resutlado em um Data Frame
    carteira = pd.DataFrame(data = resultado, columns=['Ticker', 'Quantidade', 'Preço'])

    return carteira

In [152]:
data_filtro = datetime(2022, 12, 31)

mypositions(zeca_carteira, date = data_filtro).sort_values(by="Ticker").reset_index(drop=True)

,Ticker,Quantidade,Preço
0,B3SA3,350.0,13.86
1,BBDC4,235.0,17.34
2,EGIE3,113.0,42.20
3,ENBR3,198.0,21.99
4,FLRY3,274.0,16.16
5,GGRC11,30.0,112.59
6,HAPV3,332.0,7.24
7,HGLG11,20.0,164.08
8,ITSA4,830.0,9.80
9,JBSS3,90.0,37.03
